In [1]:
import json
from openai import OpenAI
from groq import Groq
import os
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
import pickle
from requests.exceptions import HTTPError
import time
# from groq.exceptions import RateLimitError
from concurrent.futures import ThreadPoolExecutor

/home/codespace/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client =  Groq(api_key = os.environ['GROQ_API_KEY'])

In [3]:
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

/home/codespace/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
with open('../data/vietnamese_rag/documents-with-ids1.json', 'rt') as f_in:
    documents1 = json.load(f_in)

In [30]:
len(documents1)

1217

In [14]:
documents1[0].keys()

dict_keys(['group', 'context', 'question', 'answer', 'id'])

In [3]:
prompt_template = """
You emulate my assistant who works with me in a Q and A project .
Formulate 5 questions people might ask based on the record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. Make sure the questions should be in Vietnamese

The record:

question: {question}
answer: {answer}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [4]:
def generate_questions(doc):
    # Create a new dictionary excluding the first key-value pair
    doc_items = list(doc.items())[1:]  # Skip the first item
    doc_filtered = dict(doc_items)

    prompt = prompt_template.format(**doc_filtered)

    retries = 5
    for i in range(retries):
        try:
            response = client.chat.completions.create(
                model='Gemma2-9b-It',
                messages=[{"role": "user", "content": prompt}]
            )
            json_response = response.choices[0].message.content
            return json_response
        except HTTPError as e:
            if e.response.status_code == 429:  # Rate limit error
                retry_after = float(e.response.json()['error']['message'].split('in ')[-1].split('s')[0])
                time.sleep(retry_after)
            else:
                raise
        except Exception as e:
            if i < retries - 1:
                time.sleep(2 ** i)  # Exponential backoff
            else:
                raise

In [5]:
def map_progress(pool, seq, f):
    results = []

    with tqdm(total=len(seq)) as progress:
        futures = []

        for el in seq:
            future = pool.submit(f, el)
            future.add_done_callback(lambda p: progress.update())
            futures.append(future)

        for future in futures:
            result = future.result()
            results.append(result)

    return results

In [6]:

# Initialize ThreadPoolExecutor
pool = ThreadPoolExecutor(max_workers=6)

# Process documents in parallel
results = {}

In [7]:
def process_document(doc):
    doc_id = doc['id']
    if doc_id in results:
        return None

    questions = generate_questions(doc)
    return (doc_id, questions)

In [20]:
# Use map_progress to process documents
processed_results = map_progress(pool, documents1[:30], process_document)

# Store the results
for result in processed_results:
    if result is not None:
        doc_id, questions = result
        results[doc_id] = questions


  2%|█▏                                               | 28/1127 [00:20<35:37,  1.94s/it]████████████████████████████████████████████████████████████████▍          | 28/30 [00:02<00:00, 10.40it/s]

In [21]:

# Print or save the results as needed
print(results)

{'75fafd29': '[\n"Minh Tú đã gặp khó khăn gì trong thử thách đi catwalk tại Asia\'s Next Top Model mùa 5?",\n"Điểm đến nào là thử thách khó khăn nhất đối với Minh Tú trong buổi thử thách đi catwalk?",\n"Vị trí của Minh Tú trong đêm chung kết Asia\'s Next Top Model mùa 5 là gì?",\n"Minh Tú đã thể hiện kỹ năng gì khi thực hiện thử thách đi catwalk trên tòa nhà cao tầng?",\n"Những thử thách nào đã giúp Minh Tú gặt hái thành công trong Asia\'s Next Top Model mùa 5?"\n] \n', '7adda7d7': '["Tại sao TP HCM có sương mù dày đặc vào sáng hôm nay?",\n"Những yếu tố nào dẫn đến sương mù dày đặc ở TP HCM?",\n"Tại sao sương mù lại xuất hiện vào ban đêm?",\n"Có sự liên quan giữa mưa lớn và hiện tượng sương mù\xa0không?",\n"Mức độ ô nhiễm không khí có ảnh hưởng đến sương mù\xa0không?"] \n\n\n', 'de6e7aa5': '{"question1":"Vụ nổ tại nhà máy hóa dầu xảy ra ở tỉnh nào ở Trung Quốc?" , "question2":"Ảnh hưởng ban đầu của vụ nổ là bao nhiêu người bị thương?" , "question3":"Vụ nổ xảy ra vào thời gian nào theo 

In [22]:
len(processed_results)

30

In [23]:

# from collections import defaultdict

# hashes = defaultdict(list)

# for doc in documents1:
#     doc_id = doc['id']
#     hashes[doc_id].append(doc)
# # hashes['75fafd29']

In [24]:
# len = 0
# for hash in hashes:
#     print(hash)
#     len += 1
#     if len == 30:
#         break
    

In [25]:
# for result in results:
#     print(result)

In [26]:
with open('../data/vietnamese_rag/ground_truth_data/ground_truth1.pkl', 'wb') as file:
    pickle.dump(results, file)

In [27]:
with open('../data/vietnamese_rag/ground_truth_data/ground_truth1.pkl', 'rb') as file:
    test = pickle.load(file)

In [29]:
type(test)

dict

In [31]:
# Use map_progress to process documents
processed_results = map_progress(pool, documents1[30:30*2], process_document)

# Store the results
for result in processed_results:
    if result is not None:
        doc_id, questions = result
        results[doc_id] = questions


100%|█████████████████████████| 30/30 [00:03<00:00,  9.56it/s]


In [35]:
def get_last_n_items(d, n):
    # Convert dictionary items to a list
    items = list(d.items())
    # Slice the list to get the last n items
    last_n_items = items[-n:]
    # Convert the sliced list back to a dictionary
    return dict(last_n_items)

In [36]:
last_30_items = get_last_n_items(results, 30)
len(last_30_items)

30

In [37]:
with open('../data/vietnamese_rag/ground_truth_data/ground_truth2.pkl', 'wb') as file:
    pickle.dump(last_30_items, file)

In [43]:
with open('../data/vietnamese_rag/ground_truth_data/ground_truth2.pkl', 'rb') as file:
    test = pickle.load(file)

In [44]:
test

{'1549e29b': '{"questions":["Lexus NX mới có lưới tản nhiệt dạng nào?","Lexus NX mới được trang bị màn hình thông tin và giải trí có kích thước bao nhiêu inch?","NX 300h và NX 300 F Sport có điểm khác biệt nào về phần đầu xe?","Có những thay đổi ở phần đuôi xe Lexus NX mới?","Lexus NX mới sử dụng động cơ loại nào?"]}  \n',
 'f55ad056': '["Huỳnh Thị Thuỳ Dung đại diện nước nào tham dự cuộc thi nào?", "Huỳnh Thị Thuỳ Dung đạt giải gì ở cuộc thi Hoa hậu Việt Nam 2016?", "Ngoài vị trí Á hậu 2, Huỳnh Thị Thuỳ Dung còn nhận giải gì?", "Năm Huỳnh Thị Thuỳ Dung đại diện Việt Nam tham dự Miss International?", "Huỳnh Thị Thuỳ Dung là người đẹp tài năng trong cuộc thi Hoa hậu Việt Nam nào?"] \n\n\n',
 '326bd715': '["Sở dĩ Tuấn Anh không thể tham gia buổi tập đội tuyển U22 Việt Nam là do lý do gì?", "Trường hợp Đức Huy không góp mặt trong buổi tập là do chấn thương gì?", "Chuyên gia vật lý trị liệu nào đang điều trị cho Tuấn Anh?", "Cầu thủ Tuấn Anh bị chấn thương gì?", "Được biết Đức Huy bị chấn 

In [45]:
results = {}
# Use map_progress to process documents
processed_results = map_progress(pool, documents1[30*2:30*3], process_document)

# Store the results
for result in processed_results:
    if result is not None:
        doc_id, questions = result
        results[doc_id] = questions

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:02<00:00, 10.34it/s]


In [50]:
len(results)

30

In [51]:
with open('../data/vietnamese_rag/ground_truth_data/ground_truth3.pkl', 'wb') as file:
    pickle.dump(results, file)

In [10]:
results = {}
# Use map_progress to process documents
processed_results = map_progress(pool, documents1[30*3:30*4], process_document)

# Store the results
for result in processed_results:
    if result is not None:
        doc_id, questions = result
        results[doc_id] = questions

 40%|████████████████████▍                              | 12/30 [00:12<00:27,  1.53s/it]

In [11]:
with open('../data/vietnamese_rag/ground_truth_data/ground_truth4.pkl', 'wb') as file:
    pickle.dump(results, file)

In [10]:
results = {}
# Use map_progress to process documents
processed_results = map_progress(pool, documents1[30*4:30*5], process_document)

# Store the results
for result in processed_results:
    if result is not None:
        doc_id, questions = result
        results[doc_id] = questions
with open('../data/vietnamese_rag/ground_truth_data/ground_truth5.pkl', 'wb') as file:
    pickle.dump(results, file)

 17%|████████▋                                           | 5/30 [00:04<00:30,  1.20s/it]

In [11]:
results = {}
# Use map_progress to process documents
processed_results = map_progress(pool, documents1[30*5:30*6], process_document)

# Store the results
for result in processed_results:
    if result is not None:
        doc_id, questions = result
        results[doc_id] = questions
with open('../data/vietnamese_rag/ground_truth_data/ground_truth6.pkl', 'wb') as file:
    pickle.dump(results, file)

100%|█████████████████████████| 30/30 [00:13<00:00,  2.16it/s]


In [10]:
results = {}
# Use map_progress to process documents
processed_results = map_progress(pool, documents1[30*6:30*7], process_document)

# Store the results
for result in processed_results:
    if result is not None:
        doc_id, questions = result
        results[doc_id] = questions
with open('../data/vietnamese_rag/ground_truth_data/ground_truth7.pkl', 'wb') as file:
    pickle.dump(results, file)

100%|█████████████████████████| 17/17 [00:02<00:00,  7.52it/s]0 [00:03<00:00,  9.59it/s]

In [11]:
results = {}
# Use map_progress to process documents
processed_results = map_progress(pool, documents1[30*7:30*8], process_document)

# Store the results
for result in processed_results:
    if result is not None:
        doc_id, questions = result
        results[doc_id] = questions
with open('../data/vietnamese_rag/ground_truth_data/ground_truth8.pkl', 'wb') as file:
    pickle.dump(results, file)

100%|█████████████████████████| 30/30 [00:49<00:00,  1.66s/it]


In [12]:
results = {}
# Use map_progress to process documents
processed_results = map_progress(pool, documents1[30*8:30*9], process_document)

# Store the results
for result in processed_results:
    if result is not None:
        doc_id, questions = result
        results[doc_id] = questions
with open('../data/vietnamese_rag/ground_truth_data/ground_truth9.pkl', 'wb') as file:
    pickle.dump(results, file)

100%|█████████████████████████| 30/30 [00:44<00:00,  1.49s/it]


In [13]:
results = {}
# Use map_progress to process documents
processed_results = map_progress(pool, documents1[30*9:30*10], process_document)

# Store the results
for result in processed_results:
    if result is not None:
        doc_id, questions = result
        results[doc_id] = questions
with open('../data/vietnamese_rag/ground_truth_data/ground_truth10.pkl', 'wb') as file:
    pickle.dump(results, file)

100%|█████████████████████████| 30/30 [00:03<00:00,  7.54it/s]


In [14]:
results = {}
# Use map_progress to process documents
processed_results = map_progress(pool, documents1[30*10:30*11], process_document)

# Store the results
for result in processed_results:
    if result is not None:
        doc_id, questions = result
        results[doc_id] = questions
with open('../data/vietnamese_rag/ground_truth_data/ground_truth11.pkl', 'wb') as file:
    pickle.dump(results, file)

100%|███████████████████████████████████████████████████| 30/30 [00:11<00:00,  2.53it/s]


In [15]:
results = {}
# Use map_progress to process documents
processed_results = map_progress(pool, documents1[30*11:30*12], process_document)

# Store the results
for result in processed_results:
    if result is not None:
        doc_id, questions = result
        results[doc_id] = questions
with open('../data/vietnamese_rag/ground_truth_data/ground_truth12.pkl', 'wb') as file:
    pickle.dump(results, file)

100%|█████████████████████████| 30/30 [00:37<00:00,  1.25s/it]


In [16]:
results = {}
# Use map_progress to process documents
processed_results = map_progress(pool, documents1[30*12:30*13], process_document)

# Store the results
for result in processed_results:
    if result is not None:
        doc_id, questions = result
        results[doc_id] = questions
with open('../data/vietnamese_rag/ground_truth_data/ground_truth13.pkl', 'wb') as file:
    pickle.dump(results, file)

100%|█████████████████████████| 30/30 [00:23<00:00,  1.26it/s]


In [27]:
# Process documents in chunks of 30
chunk_size = 30
start_chunk = 13  # Starting chunk index
end_chunk = (len(documents1) // chunk_size)  # Ending chunk index
print(end_chunk)
for i in range(start_chunk, end_chunk):
    results = {}
    chunk_start = i * chunk_size
    chunk_end = chunk_start + chunk_size
    chunk = documents1[chunk_start:chunk_end]
    print(chunk_start, chunk_end)

#     # Use map_progress to process documents
#     processed_results = map_progress(pool, chunk, process_document)

#     # Store the results incrementally
#     for result in processed_results:
#         if result is not None:
#             doc_id, questions = result
#             results[doc_id] = questions

#     # Save the results to a file
#     file_name = f'../data/vietnamese_rag/ground_truth_data/ground_truth{i}.pkl'
#     with open(file_name, 'wb') as file:
#         pickle.dump(results, file)

#     # Print out the results
#     print(f"Chunk {i} processed and saved to {file_name}")
#     print(results)

#     # Wait for 1 minute to reset rate limit
#     time.sleep(120)

40
390 420
420 450
450 480
480 510
510 540
540 570
570 600
600 630
630 660
660 690
690 720
720 750
750 780
780 810
810 840
840 870
870 900
900 930
930 960
960 990
990 1020
1020 1050
1050 1080
1080 1110
1110 1140
1140 1170
1170 1200


In [30]:
# Process documents in chunks of 30
chunk_size = 30
start_chunk = 14  # Starting chunk index
end_chunk = (len(documents1) // chunk_size)  # Ending chunk index
print(end_chunk)
for i in range(start_chunk, end_chunk):
    results = {}
    chunk_start = i * chunk_size
    chunk_end = chunk_start + chunk_size
    chunk = documents1[chunk_start:chunk_end]

    # Use map_progress to process documents
    processed_results = map_progress(pool, chunk, process_document)

    # Store the results incrementally
    for result in processed_results:
        if result is not None:
            doc_id, questions = result
            results[doc_id] = questions

    # Save the results to a file
    file_name = f'../data/vietnamese_rag/ground_truth_data/ground_truth{i + 1}.pkl'
    with open(file_name, 'wb') as file:
        pickle.dump(results, file)

    # Print out the results
    print(f"Chunk {i} processed and saved to {file_name}")
    print(results)

    # Wait for 1 minute to reset rate limit
    time.sleep(60)

40


100%|█████████████████████████| 30/30 [00:03<00:00,  9.24it/s]


Chunk 14 processed and saved to ../data/vietnamese_rag/ground_truth_data/ground_truth15.pkl
{'96911e78': '[\n"Mũ bucket trở thành xu hướng thời trang đường phố vào những năm nào?",\n"Ai là nghệ sĩ  đã làm cho mũ bucket trở nên nổi tiếng?",\n"LL Cool J có liên quan gì đến mũ bucket?",\n"Năm 1980 ấn tượng với giới trẻ thế nào qua mũ bucket?",\n"Mũ bucket có một vai trò quan trọng trong phong cách thời trang như thế nào?" \n] \n', 'f3f96cd5': '["Renee hoạt động nghề nghiệp gì ở thành phố St Kilda?",\n"Renee được nổi tiếng như thế nào ở thành phố St Kilda?",\n"Theo ghi nhận, ai là người được xem là gái đẹp nhất tại St Kilda?",\n"Thành phố nào là nơi Renee làm việc?",\n"Nơi nào Renee được công nhận là gái đẹp nhất?"] \n', '11b044a1': '["Số bàn thắng MU ghi được trong trận đấu với Basel năm 2017/2018?", "Điểm số của trận đấu MU vs Basel vòng đấu đầu tiên Champions League 2017/2018?", "Tỷ số trận đấu đầu tiên MU gặp Basel tại Champions League  là bao nhiêu?", "Kết quả trận đấu giữa MU và Base

100%|█████████████████████████| 30/30 [00:03<00:00,  8.30it/s]


Chunk 15 processed and saved to ../data/vietnamese_rag/ground_truth_data/ground_truth16.pkl
{'4952f80b': '["M.U đã thi đấu với những đội bóng nào trong vòng bảng Champions League mùa 2017/18?", "M.U nằm ở bảng nào của vòng bảng Champions League mùa 2017/18?", "Quels sont les adversaires de M.U au cours de la phase de groupes de la Ligue des Champions 2017/18?", "M.U đã có những đối thủ nào ở Cúp C1 mùa 2017/18?",] \n', '57fcd2eb': '[\n"Chính sách \'ở nguyên tại chỗ\' trong trường hợp hỏa hoạn được xem xét lại sau vụ cháy nào?",\n"Theo ông Darren Baird, đối tượng nào cần được xem xét sát sao về chính sách \'ở nguyên tại chỗ\'?",\n"Tại sao lời khuyên \'ở yên tại chỗ\' có thể trở nên phản tác dụng trong trường hợp hỏa hoạn?",\n"Ông Darren Baird là ai?",\n"Vai trò của ông Darren Baird trong lĩnh vực an toàn cháy nổ là gì?"\n] \n', 'e0b32951': '["Thực chất một đối tượng đã làm gì khi bị cảnh sát áp phục ?", \n"Bagaimana cara seorang tersangka menghindari penangkapan dari polisi?", \n"¿Qué h

...

Chunk 37 processed and saved to ../data/vietnamese_rag/ground_truth_data/ground_truth38.pkl
{'bc84d744': '["Trận chung kết U23 châu Á diễn ra giữa U23 Việt Nam và đội tuyển nào?", \n"Bùi Tiến Dũng đã có màn trình diễn như thế nào trong trận chung kết U23 châu Á?",\n"Thành tích cụ thể nào mà Bùi Tiến Dũng đã giúp U23 Việt Nam đạt được?",\n"Tại sao U23 Việt Nam tiến thẳng vào vòng chung kết U23 châu Á?",\n"Những đóng góp quan trọng của Bùi Tiến Dũng bao gồm những gì?"] \n', 'c083b599': '["Những quyền lợi thể chế nào Anh quốc sẽ mất sau Brexite?", "EU có thể yêu cầu Anh tuân thủ luật pháp của EU như thế nào sau Brexit?", "Anh quốc sẽ có vai trò như thế nào trong các tổ chức EU sau Brexit?", "Liệu Anh quốc có được quyền biểu quyết khi EU ban hành luật mới cho các nước thành viên?", "EU có thể cung cấp những điều kiện tài chính thuận lợi nào cho Anh sau Brexit?"]  \n', 'ddeccc5b': '[\n"Trong 6 tháng đầu năm, nước nào có tổng vốn đầu tư vào Việt Nam cao nhất?",\n"Hàn Quốc đã đầu tư bao nhiêu

100%|█████████████████████████| 30/30 [00:03<00:00,  8.62it/s]


Chunk 38 processed and saved to ../data/vietnamese_rag/ground_truth_data/ground_truth39.pkl
{'4b124b46': '["Số lượng đối tượng bị bắt liên quan đến hoạt động kinh doanh cờ bạc trá hình trong trung tâm game bắn cá là bao nhiêu?", "Ngoài những người bị bắt, còn có bao nhiêu con bạc tham gia vào hoạt động này?", "Theo thông tin, đã có bao nhiêu người bị bắt vì tội kinh doanh cờ bạc trá hình trong lĩnh vực game bắn cá?", "Có bao nhiêu con bạc được xác định có liên quan đến vụ việc kinh doanh cờ bạc trá hình?", "Vụ việc kinh doanh cờ bạc trá hình tại trung tâm game bắn cá đã bắt giữ bao nhiêu người?"]   \n', '86471ba9': '{"questions": [\n"Vi phạm giao thông nào mà Công an xã được phép xử lý?",\n"Ngoài việc xử phạt không mang bằng lái, các hành vi giao thông nào Công an xã có thẩm quyền xử lý?",\n"Những trường hợp nào cho phép Công an xã xử phạt hành vi vi phạm an toàn giao thông?",\n"Công an xã có quyền xử phạt hành vi nào liên quan đến mó tô, xe gắn máy?",\n"Hành vi nào dưới lòng đường sẽ 

100%|█████████████████████████| 30/30 [00:03<00:00,  8.24it/s]


Chunk 39 processed and saved to ../data/vietnamese_rag/ground_truth_data/ground_truth40.pkl
{'3619358f': '["Vi phạm sử dụng vốn công ty nhà nước có thể bị phạt bao nhiêu?", "Linh vực phạt tiền đối với hành vi lãng phí  vốn công ty nhà nước là gì?", "Bộ luật nào quy định về phạt tiền cho hành vi lãng phí vốn công ty nhà nước?", "Hạn mức nào được áp dụng với hành vi lãng phí vốn công ty nhà nước?", "Mức phạt tiền tối đa cho hành vi lãng phí vốn công ty nhà nước là bao nhiêu?"] \n', '924427ec': '["Hội đồng đạo đức cấp cơ sở cần phải gửi hồ sơ đề nghị cập nhật thông tin đến cơ quan nào?", "Thời hạn gửi hồ sơ đề nghị cập nhật thông tin đến cơ quan liên quan là bao nhiêu ngày?", "Cục Khoa học Công nghệ và Đào tạo có trách nhiệm đối với hồ sơ đề nghị cập nhật thông tin của Hội đồng đạo đức trong thời hạn bao nhiêu ngày?", "Điều kiện để Cục Khoa học Công nghệ và Đào tạo cập nhật thông tin của Hội đồng đạo đức trên Trang thông tin điện tử của mình là gì?", "Hội đồng đạo đức cấp cơ sở cần làm gì

In [32]:
results = {}
# Use map_progress to process documents
processed_results = map_progress(pool, documents1[1200:], process_document)

# Store the results
for result in processed_results:
    if result is not None:
        doc_id, questions = result
        results[doc_id] = questions
with open('../data/vietnamese_rag/ground_truth_data/ground_truth41.pkl', 'wb') as file:
    pickle.dump(results, file)

100%|█████████████████████████| 17/17 [00:02<00:00,  7.63it/s]


In [33]:
with open('../data/vietnamese_rag/documents-with-ids2.json', 'rt') as f_in:
    documents2 = json.load(f_in)

In [34]:
len(documents2)

1217

In [39]:
chunk_size = 30
start_chunk = 0 # Starting chunk index
end_chunk = (len(documents2) // chunk_size)  # Ending chunk index
print(end_chunk)
for i in range(start_chunk, end_chunk):
    results = {}
    chunk_start = i * chunk_size
    chunk_end = chunk_start + chunk_size
    print(i + 42, chunk_start, chunk_end)
    # chunk = documents1[chunk_start:chunk_end]

40
42 0 30
43 30 60
44 60 90
45 90 120
46 120 150
47 150 180
48 180 210
49 210 240
50 240 270
51 270 300
52 300 330
53 330 360
54 360 390
55 390 420
56 420 450
57 450 480
58 480 510
59 510 540
60 540 570
61 570 600
62 600 630
63 630 660
64 660 690
65 690 720
66 720 750
67 750 780
68 780 810
69 810 840
70 840 870
71 870 900
72 900 930
73 930 960
74 960 990
75 990 1020
76 1020 1050
77 1050 1080
78 1080 1110
79 1110 1140
80 1140 1170
81 1170 1200


In [40]:
chunk_size = 30
start_chunk = 0 # Starting chunk index
end_chunk = (len(documents2) // chunk_size)  # Ending chunk index
print(end_chunk)
for i in range(start_chunk, end_chunk):
    results = {}
    chunk_start = i * chunk_size
    chunk_end = chunk_start + chunk_size
    # print(i + 42, chunk_start, chunk_end)
    chunk = documents2[chunk_start:chunk_end]

    # Use map_progress to process documents
    processed_results = map_progress(pool, chunk, process_document)

    # Store the results incrementally
    for result in processed_results:
        if result is not None:
            doc_id, questions = result
            results[doc_id] = questions

    # Save the results to a file
    file_name = f'../data/vietnamese_rag/ground_truth_data/ground_truth{i + 42}.pkl'
    with open(file_name, 'wb') as file:
        pickle.dump(results, file)

    # Print out the results
    print(f"Chunk {i} processed and saved to {file_name}")
    print(results)

    # Wait for 1 minute to reset rate limit
    time.sleep(60)

40


100%|███████████████████████████████████████████████████| 30/30 [00:03<00:00,  8.24it/s]


Chunk 0 processed and saved to ../data/vietnamese_rag/ground_truth_data/ground_truth42.pkl
{'a37a1821': '["Hành vi điều khiển xe ô tô với giấy đăng ký hết hạn, phạt bao nhiêu tiền?",\n"Tại sao lại bị phạt khi điều khiển ô tô với giấy đăng ký xe hết hạn?",\n"Có mức phạt cụ thể nào cho lỗi sử dụng giấy đăng ký xe hết hạn?",\n"Những trường hợp nào bị phạt tiền từ 2 đến 3 triệu đồng khi lái xe?",\n"Mức phạt tiền cho việc sử dụng giấy đăng ký xe hết hạn nhiều hơn bao nhiêu?"] \n', '3e81d992': '["Sau 18 tháng làm việc tại bệnh viện, ở đâu bác sĩ có thể hành nghề?", "Thời gian thực hành tại bệnh viện để được cấp chứng chỉ hành nghề bao nhiêu tháng?", "Yêu cầu về thời gian thực hành để được cấp chứng chỉ hành nghề cho bác sĩ là gì?", "Bác sĩ có thể nhận được giấy chứng nhận hành nghề khám chữa bệnh nếu thực hành tại phòng khám tư không?", "Đối với bác sĩ mới tốt nghiệp, điều kiện để được cấp chứng chỉ hành nghề là gì?"] \n', '981d0211': '["Hình thức kỷ luật nào được áp dụng đối với quân nhân v

100%|█████████████████████████| 30/30 [00:03<00:00,  9.19it/s]


Chunk 1 processed and saved to ../data/vietnamese_rag/ground_truth_data/ground_truth43.pkl
{'641489d8': '[\n"Hội đồng định giá được gửi Quyết định định giá tài sản và các tài liệu liên quan theo thời hạn bao lâu?",\n"Tòa án gửi Quyết định định giá tài sản và tài liệu liên quan đến ai?",\n"Trong trường hợp nào, Ủy ban nhân dân xã, phường, thị trấn được mời tham gia chứng kiến việc định giá?",\n"Tòa án gửi Quyết định định giá tài sản và các tài liệu liên quan cho các bên tham gia định giá bao gồm ai?",\n"Kể từ ngày ra Quyết định định giá tài sản, Tòa án gửi Quyết định định giá và các tài liệu liên quan trong bao lâu?"\n]\n\n\n\n\n', '1200a38b': '[\n"Hồ sơ kết quả pháp điển hệ thống quy phạm pháp luật gửi thẩm định bao gồm những gì?",\n"Ai là người ký xác thực kết quả pháp điển theo đề mục?",\n"Khi nào cơ quan thực hiện pháp điển bổ sung kết quả pháp điển và gửi hồ sơ bổ sung đến Bộ Tư pháp?",\n"Theo văn bản nào khung thời hạn để gửi hồ sơ kết quả pháp điển?",\n"Nếu có văn bản được ban hà

...

Chunk 37 processed and saved to ../data/vietnamese_rag/ground_truth_data/ground_truth79.pkl
{'ecf9beeb': '["Chứng từ GSP có giá trị hiệu lực bao lâu?", "Thời hạn sử dụng của giấy chứng nhận xuất xứ hàng hóa GSP là bao nhiêu?", "Họp lực của văn bản chứng nhận xuất xứ hàng hóa theo GSP kéo dài trong bao lâu?", "Ngày phát hành chứng minh GSP có ý nghĩa gì đối với độ dài hiệu lực?", "Giấy chứng nhận xuất xứ GSP có hiệu lực trong bao lâu kể từ ngày được cấp?" ] \n', '8d4400e2': '["Bạn có thể cho biết mức phạt cho việc thi công công trình trên đất dành cho đường bộ mà đã hết hạn Giấy phép thi công không?", "Việc thi công công trình vượt quá thời hạn Giấy phép thi công sẽ bị phạt bao nhiêu tiền?", "Nếu thi công công trình trên đất đường bộ lỗi thời hạn Giấy phép thi công, cá nhân và tổ chức sẽ bị phạt bao nhiêu?", "Mức phạt cho việc thi công trái phép trên đất đường bộ liệu có khác nhau giữa cá nhân và tổ chức?", "Linh vực nào áp dụng mức phạt tiền từ 3.000.000 đồng đến 5.000.000 đồng đối với

100%|█████████████████████████| 30/30 [00:04<00:00,  6.61it/s]


Chunk 38 processed and saved to ../data/vietnamese_rag/ground_truth_data/ground_truth80.pkl
{'ec587a7c': '{"question1": "Quy trình kiểm tra giấy tờ tùy thân như thế nào khi đến khách sạn thăm bạn?", "question2": "Khách sạn cần lưu trữ thông tin về ai?", "question3": "Bao nhiêu thời gian khách sạn phải lưu giữ thông tin của người đến thăm?","question4": "Khi ra khỏi khách sạn, liệu người đến thăm có được lấy lại giấy tờ tùy thân?", "question5": "Những thông tin nào của người đến thăm được ghi vào sổ?"} \n', 'd2c7880b': '[\n"Theo Luật Cạnh tranh, việc không thông báo khi tập trung kinh tế sẽ bị xử phạt như thế nào?",\n"Giải thích về hình phạt đối với doanh nghiệp vi phạm nghĩa vụ thông báo khi tập trung kinh tế.",\n"Hạn mức phạt tiền đối với doanh nghiệp vi phạm Điều 33 của Luật Cạnh tranh là bao nhiêu?",\n"Tính toán phạt tiền đối với vi phạm Điều 33 Luật Cạnh tranh dựa trên cơ sở gì?", \n"Tại sao doanh nghiệp cần phải thông báo khi tập trung kinh tế?"\n] \n\n\n', 'cae22d60': '["Người có

100%|█████████████████████████| 30/30 [00:03<00:00,  7.61it/s]


Chunk 39 processed and saved to ../data/vietnamese_rag/ground_truth_data/ground_truth81.pkl
{'201d284c': '["Theo quy định, hành vi nào sẽ bị xử lý như không có giấy đăng ký xe?", "Điều kiện để bị lập biên bản vi phạm hành chính khi điều khiển phương tiện là gì?", "Nghị định nào quy định về xử lý hành vi không xuất trình được giấy đăng ký xe?", "Hành vi không xuất trình được giấy đăng ký xe có thể dẫn đến hậu quả gì?", "Phương tiện nào phải xuất trình giấy đăng ký xe khi điều khiển?"] \n', 'd2345f1f': '["Cơ quan nhà nước có thẩm quyền yêu cầu thông tin về an toàn thực phẩm nhưng người kinh doanh không cung cấp, phạt bao nhiêu?", "Vi phạm cung cấp thông tin an toàn thực phẩm, mức phạt tiền cụ thể là bao nhiêu?", "Mức phạt tiền tối thiểu khi không cung cấp thông tin an toàn thực phẩm theo yêu cầu cơ quan nhà nước là bao nhiêu?", "Giá trị phạt tiền cho việc không cung cấp thông tin an toàn thực phẩm như yêu cầu có giới hạn nào không?", "Hình phạt tài chính đối với việc không tuân thủ yêu c

In [41]:
results = {}
# Use map_progress to process documents
processed_results = map_progress(pool, documents2[1200:], process_document)

# Store the results
for result in processed_results:
    if result is not None:
        doc_id, questions = result
        results[doc_id] = questions
with open('../data/vietnamese_rag/ground_truth_data/ground_truth82.pkl', 'wb') as file:
    pickle.dump(results, file)

100%|█████████████████████████| 17/17 [00:05<00:00,  3.09it/s]


In [42]:
with open('../data/vietnamese_rag/documents-with-ids3.json', 'rt') as f_in:
    documents3 = json.load(f_in)

In [43]:
chunk_size = 30
start_chunk = 0 # Starting chunk index
end_chunk = (len(documents3) // chunk_size)  # Ending chunk index
print(end_chunk)
for i in range(start_chunk, end_chunk):
    results = {}
    chunk_start = i * chunk_size
    chunk_end = chunk_start + chunk_size
    # print(i + 42, chunk_start, chunk_end)
    chunk = documents3[chunk_start:chunk_end]

    # Use map_progress to process documents
    processed_results = map_progress(pool, chunk, process_document)

    # Store the results incrementally
    for result in processed_results:
        if result is not None:
            doc_id, questions = result
            results[doc_id] = questions

    # Save the results to a file
    file_name = f'../data/vietnamese_rag/ground_truth_data/ground_truth{i + 83}.pkl'
    with open(file_name, 'wb') as file:
        pickle.dump(results, file)

    # Print out the results
    print(f"Chunk {i} processed and saved to {file_name}")
    print(results)

    # Wait for 1 minute to reset rate limit
    time.sleep(60)

40


100%|█████████████████████████| 30/30 [00:41<00:00,  1.39s/it]


Chunk 0 processed and saved to ../data/vietnamese_rag/ground_truth_data/ground_truth83.pkl
{'18dae9ff': '{"Sách Quy chi": "Chi phí lập đồ án quy hoạch chung đô thị được xác định như thế nào?",\n "Chi phí": "Các khoản chi phí nào được bao gồm trong chi phí lập đồ án quy hoạch?",\n "Quy mô": "Người ta dựa vào những yếu tố nào để xác định chi phí lập đồ án quy hoạch?",\n "Phụ lục 1": "Ở đâu người ta có thể tìm thấy các bảng định mức chi phí quy hoạch?",\n "Tỷ lệ": "Đối với thiết kế đô thị riêng, chi phí được tính theo cách nào?"} \n', '566d2af1': '["Địa phương có vai trò như thế nào trong việc tổ chức hướng dẫn kỹ thuật Hệ thống cơ sở dữ liệu ngành tại các cơ sở giáo dục?", "Những trường hợp nào đòi hỏi địa phương phải báo cáo sự cố kỹ thuật liên quan đến Hệ thống cơ sở dữ liệu ngành?", "Ai là đơn vị nhận báo cáo về sự cố kỹ thuật liên quan đến Hệ thống cơ sở dữ liệu ngành?", "Cục nào của Bộ Giáo dục và Đào tạo xử lý các báo cáo về sự cố kỹ thuật liên quan đến Hệ thống?", "Nhà quản lý hệ 

100%|█████████████████████████| 30/30 [00:11<00:00,  2.67it/s]


Chunk 1 processed and saved to ../data/vietnamese_rag/ground_truth_data/ground_truth84.pkl
{'fd882ec2': '[\n"Theo quy định pháp luật nào, việc bảo trì tài sản kết cấu hạ tầng thủy lợi được thực hiện?",\n"Ngoài giai đoạn sau khi bàn giao, bảo trì công trình hạ tầng thủy lợi bao gồm giai đoạn nào?",\n"Khi bảo trì, cần đảm bảo điều gì cho công trình và các thiết bị liên quan?",\n"Trong quá trình bảo trì, việc thay đổi mục đích sử dụng của công trình được phép hay không?",\n"Trong quá trình bảo trì, cần hạn chế ảnh hưởng đến điều gì của công trình thủy lợi?"\n] \n\n\n', '0fd566f5': '["Số tiền phạt tối đa dành cho tổ chức vi phạm hành chính trong lĩnh vực khai thác dầu khí là bao nhiêu?", "Theo luật, mức phạt tiền tối đa đối với tổ chức vi phạm tại ngành dầu khí là bao nhiêu?", "Mức phạt cao nhất đối với tổ chức vi phạm hành chính trong thăm dò, khai thác dầu khí là bao nhiêu?", "Đối với vi phạm hành chính của tổ chức trong lĩnh vực thăm dò, khai thác dầu khí, mức phạt tiền có thể lên tới b

...

Chunk 37 processed and saved to ../data/vietnamese_rag/ground_truth_data/ground_truth120.pkl
{'a4bb327d': '[\n"Người dưới 18 tuổi có được gọi điện cho gia đình không?",\n"Giới hạn lần gọi điện của phạm nhân dưới 18 tuổi là bao nhiêu trong một tháng?",\n"Mỗi cuộc gọi điện của phạm nhân dưới 18 tuổi được kéo dài tối đa bao nhiêu phút?",\n"Phạm nhân dưới 18 tuổi có được phép liên lạc với gia đình bằng điện thoại không?",\n"Ai giám sát cuộc gọi điện của phạm nhân dưới 18 tuổi?"\n]  \n\n\n', '0de9a7fe': '{"questions": ["Nhà thầu thi công xây dựng và nhà thầu cung ứng thiết bị cần làm gì để công nhận việc bảo hành công trình hoàn tất?", "Chủ đầu tư có những trách nhiệm gì khi xác nhận hoàn thành bảo hành công trình xây dựng?", "Điều kiện để chủ đầu tư hoàn trả tiền bảo hành cho nhà thầu là gì?", "Ai có trách nhiệm tham gia xác nhận hoàn thành bảo hành công trình xây dựng khi chủ đầu tư yêu cầu?", "Văn bản chủ đầu tư sử dụng để xác nhận hoàn thành bảo hành công trình như thế nào?"]} \n', 'd8c

100%|█████████████████████████| 30/30 [00:03<00:00,  9.59it/s]


Chunk 38 processed and saved to ../data/vietnamese_rag/ground_truth_data/ground_truth121.pkl
{'b0a7c303': '[\n"Thông tin về người sở hữu chứng khoán được ghi lại ở đâu?",\n"Ai có quyền quản lý việc đăng ký chứng khoán?",\n"Tổng công ty lưu ký và bù trừ chứng khoán Việt Nam có vai trò gì trong việc quản lý chứng khoán?",\n"Theo bạn, chứng khoán được bán cho ai?" ,\n"Điều kiện để được ghi tên trên sổ đăng ký sở hữu chứng khoán là gì?"\n] \n', 'f1590119': '{"questions":["Nguồn ngân sách nào được sử dụng để hỗ trợ phát triển nhân lực DNNVV?", "Chính sách hỗ trợ DNNVV được tài trợ bởi ngân sách trung ương theo hình thức nào?", "Biên chế ngân sách địa phương được bố trí để hỗ trợ phát triển nguồn nhân lực DNNVV ở đâu?", "Bộ nào, cơ quan nào trong ngân sách trung ương được giao nhiệm vụ hỗ trợ phát triển nhân lực DNNVV?", "Ngoài ngân sách trung ương và địa phương, có những nguồn hỗ trợ nào khác cho DNNVV?"]} \n', 'c48b9e9e': '["Những loài thủy sản nào được cấm xuất khẩu?", "Xuất khẩu giống th

100%|█████████████████████████| 30/30 [00:03<00:00,  9.34it/s]


Chunk 39 processed and saved to ../data/vietnamese_rag/ground_truth_data/ground_truth122.pkl
{'d0720e42': '["Nghiên cứu đào tạo lái xe có công khai quy chế tuyển sinh không?", "Cơ sở đào tạo lái xe phải công khai thông tin gì?", "Theo quy định, cơ sở đào tạo lái xe có bị ràng buộc về công khai học phí không?", "Mức thu học phí ở cơ sở đào tạo lái xe được quản lý như thế nào?", "Tại sao cơ sở đào tạo lái xe cần công khai thông tin?"] \n', 'e785eac6': '[\n"Hướng dẫn nào để nộp hồ sơ công nhận tổ chức đủ điều kiện tổ chức sát hạch cấp CCHN kiến trúc?",\n"Những gì cần có trong đơn đề nghị công nhận đủ điều kiện tổ chức sát hạch kiến trúc?",\n"Bản thuyết minh về khả năng đáp ứng yêu cầu tại khoản 3 Điều 26 Luật Kiến trúc cần có nội dung gì?",\n"Loại văn bản nào được yêu cầu chứng thực khả năng hoạt động của tổ chức?",\n"Người liên hệ để nhận được mẫu đơn đề nghị công nhận đủ điều kiện tổ chức sát hạch kiến trúc là ai?"\n] \n', 'bdcad92d': '[\n"Từ tháng 7/2020, trường của cơ quan nhà nước vẫ

In [46]:
# results = {}
# # Use map_progress to process documents
# processed_results = map_progress(pool, documents3[1200:], process_document)

# # Store the results
# for result in processed_results:
#     if result is not None:
#         doc_id, questions = result
#         results[doc_id] = questions
# with open('../data/vietnamese_rag/ground_truth_data/ground_truth84.pkl', 'wb') as file:
#     pickle.dump(results, file)

In [47]:
# with open('../data/vietnamese_rag/documents-with-ids4.json', 'rt') as f_in:
#     documents4 = json.load(f_in)

In [48]:
# chunk_size = 30
# start_chunk = 0 # Starting chunk index
# end_chunk = (len(documents4) // chunk_size)  # Ending chunk index
# print(end_chunk)
# for i in range(start_chunk, end_chunk):
#     results = {}
#     chunk_start = i * chunk_size
#     chunk_end = chunk_start + chunk_size
#     # print(i + 42, chunk_start, chunk_end)
#     chunk = documents4[chunk_start:chunk_end]

#     # Use map_progress to process documents
#     processed_results = map_progress(pool, chunk, process_document)

#     # Store the results incrementally
#     for result in processed_results:
#         if result is not None:
#             doc_id, questions = result
#             results[doc_id] = questions

#     # Save the results to a file
#     file_name = f'../data/vietnamese_rag/ground_truth_data/ground_truth{i + 85}.pkl'
#     with open(file_name, 'wb') as file:
#         pickle.dump(results, file)

#     # Print out the results
#     print(f"Chunk {i} processed and saved to {file_name}")
#     print(results)

#     # Wait for 1 minute to reset rate limit
#     time.sleep(60)

In [49]:
results = {}
# Use map_progress to process documents
processed_results = map_progress(pool, documents3[1200:], process_document)

# Store the results
for result in processed_results:
    if result is not None:
        doc_id, questions = result
        results[doc_id] = questions
with open('../data/vietnamese_rag/ground_truth_data/ground_truth123.pkl', 'wb') as file:
    pickle.dump(results, file)

100%|███████████████████████████████████████████████████| 17/17 [00:02<00:00,  6.71it/s]


In [50]:
with open('../data/vietnamese_rag/documents-with-ids4.json', 'rt') as f_in:
    documents4 = json.load(f_in)

In [51]:
chunk_size = 30
start_chunk = 0 # Starting chunk index
end_chunk = (len(documents4) // chunk_size)  # Ending chunk index
print(end_chunk)
for i in range(start_chunk, end_chunk):
    results = {}
    chunk_start = i * chunk_size
    chunk_end = chunk_start + chunk_size
    # print(i + 42, chunk_start, chunk_end)
    chunk = documents4[chunk_start:chunk_end]

    # Use map_progress to process documents
    processed_results = map_progress(pool, chunk, process_document)

    # Store the results incrementally
    for result in processed_results:
        if result is not None:
            doc_id, questions = result
            results[doc_id] = questions

    # Save the results to a file
    file_name = f'../data/vietnamese_rag/ground_truth_data/ground_truth{i + 124}.pkl'
    with open(file_name, 'wb') as file:
        pickle.dump(results, file)

    # Print out the results
    print(f"Chunk {i} processed and saved to {file_name}")
    # print(results)

    # Wait for 1 minute to reset rate limit
    time.sleep(60)

40


100%|█████████████████████████| 30/30 [00:14<00:00,  2.04it/s]


Chunk 0 processed and saved to ../data/vietnamese_rag/ground_truth_data/ground_truth124.pkl


100%|█████████████████████████| 30/30 [00:03<00:00,  8.87it/s]


Chunk 1 processed and saved to ../data/vietnamese_rag/ground_truth_data/ground_truth125.pkl


...

Chunk 37 processed and saved to ../data/vietnamese_rag/ground_truth_data/ground_truth161.pkl


100%|█████████████████████████| 30/30 [01:08<00:00,  2.28s/it]


Chunk 38 processed and saved to ../data/vietnamese_rag/ground_truth_data/ground_truth162.pkl


100%|█████████████████████████| 30/30 [02:24<00:00,  4.81s/it]


Chunk 39 processed and saved to ../data/vietnamese_rag/ground_truth_data/ground_truth163.pkl


In [52]:
chunk_size = 30
start_chunk = 0 # Starting chunk index
end_chunk = (len(documents4) // chunk_size)  # Ending chunk index
print(end_chunk)
for i in range(start_chunk, end_chunk):
    results = {}
    chunk_start = i * chunk_size
    chunk_end = chunk_start + chunk_size
    print(i + 124, chunk_start, chunk_end)

40
124 0 30
125 30 60
126 60 90
127 90 120
128 120 150
129 150 180
130 180 210
131 210 240
132 240 270
133 270 300
134 300 330
135 330 360
136 360 390
137 390 420
138 420 450
139 450 480
140 480 510
141 510 540
142 540 570
143 570 600
144 600 630
145 630 660
146 660 690
147 690 720
148 720 750
149 750 780
150 780 810
151 810 840
152 840 870
153 870 900
154 900 930
155 930 960
156 960 990
157 990 1020
158 1020 1050
159 1050 1080
160 1080 1110
161 1110 1140
162 1140 1170
163 1170 1200


In [53]:
results = {}
# Use map_progress to process documents
processed_results = map_progress(pool, documents4[1200:], process_document)

# Store the results
for result in processed_results:
    if result is not None:
        doc_id, questions = result
        results[doc_id] = questions
with open('../data/vietnamese_rag/ground_truth_data/ground_truth164.pkl', 'wb') as file:
    pickle.dump(results, file)

100%|█████████████████████████| 17/17 [00:02<00:00,  6.55it/s]


In [8]:
with open('../data/vietnamese_rag/documents-with-ids5.json', 'rt') as f_in:
    documents5 = json.load(f_in)

In [9]:
len(documents5)

1221

In [10]:
chunk_size = 30
start_chunk = 0 # Starting chunk index
end_chunk = (len(documents5) // chunk_size)  # Ending chunk index
print(end_chunk)
for i in range(start_chunk, end_chunk):
    results = {}
    chunk_start = i * chunk_size
    chunk_end = chunk_start + chunk_size
    print(i + 165, chunk_start, chunk_end)

40
165 0 30
166 30 60
167 60 90
168 90 120
169 120 150
170 150 180
171 180 210
172 210 240
173 240 270
174 270 300
175 300 330
176 330 360
177 360 390
178 390 420
179 420 450
180 450 480
181 480 510
182 510 540
183 540 570
184 570 600
185 600 630
186 630 660
187 660 690
188 690 720
189 720 750
190 750 780
191 780 810
192 810 840
193 840 870
194 870 900
195 900 930
196 930 960
197 960 990
198 990 1020
199 1020 1050
200 1050 1080
201 1080 1110
202 1110 1140
203 1140 1170
204 1170 1200


In [ ]:
chunk_size = 30
start_chunk = 0 # Starting chunk index
end_chunk = (len(documents5) // chunk_size)  # Ending chunk index
print(end_chunk)
for i in range(start_chunk, end_chunk):
    results = {}
    chunk_start = i * chunk_size
    chunk_end = chunk_start + chunk_size
    # print(i + 42, chunk_start, chunk_end)
    chunk = documents5[chunk_start:chunk_end]

    # Use map_progress to process documents
    processed_results = map_progress(pool, chunk, process_document)

    # Store the results incrementally
    for result in processed_results:
        if result is not None:
            doc_id, questions = result
            results[doc_id] = questions

    # Save the results to a file
    file_name = f'../data/vietnamese_rag/ground_truth_data/ground_truth{i + 165}.pkl'
    with open(file_name, 'wb') as file:
        pickle.dump(results, file)

    # Print out the results
    print(f"Chunk {i} processed and saved to {file_name}")
    # print(results)

    # Wait for 1 minute to reset rate limit
    time.sleep(30)

40


100%|███████████████████████████| 30/30 [00:18<00:00,  1.64it/s]


Chunk 0 processed and saved to ../data/vietnamese_rag/ground_truth_data/ground_truth165.pkl


100%|██████████████████████████████████████████████████████████| 30/30 [00:40<00:00,  1.36s/it]


Chunk 1 processed and saved to ../data/vietnamese_rag/ground_truth_data/ground_truth166.pkl


...

100%|███████████████████████████████| 30/30 [00:51<00:00,  1.71s/it]


Chunk 8 processed and saved to ../data/vietnamese_rag/ground_truth_data/ground_truth173.pkl


100%|███████████████████████████████| 30/30 [00:46<00:00,  1.56s/it]


Chunk 9 processed and saved to ../data/vietnamese_rag/ground_truth_data/ground_truth174.pkl


 87%|██████████████████████████▊    | 26/30 [00:32<00:09,  2.41s/it]

In [ ]:
results = {}
# Use map_progress to process documents
processed_results = map_progress(pool, documents5[1200:], process_document)

# Store the results
for result in processed_results:
    if result is not None:
        doc_id, questions = result
        results[doc_id] = questions
with open('../data/vietnamese_rag/ground_truth_data/ground_truth205.pkl', 'wb') as file:
    pickle.dump(results, file)